### Import data

In [1]:
#These are store after running the "Data preprocessing" notebook

%store -r df_qsar
%store -r X_train          
%store  -r X_test  
%store -r y_train
%store -r  y_test


In [2]:
%store -r X
%store -r X_df
%store -r log_y

In [3]:
print (X.shape)
print (log_y.shape)

(299, 736)
(299, 3)


In [4]:
from sklearn import metrics
from sklearn import svm
from sklearn import pipeline
from sklearn import datasets, linear_model
from sklearn import model_selection
from math import sqrt

from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn import linear_model



In [5]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
conda install scikit-learn==0.21
from genetic_selection import GeneticSelectionCV

#If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+
#I need to modify the GeneticSelectionCV code. 

SyntaxError: invalid syntax (<ipython-input-8-5c22210b8388>, line 1)

### Define mean squared error as the score 
###### !Later the sqrt of the score will be calculated

In [ ]:
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)   

###  Define genetic algorithm as the selector and fit

In [ ]:
pipe_DCAA_rf = Pipeline(
    [
        ('std_scaler', StandardScaler()),
        ("RF", RandomForestRegressor(random_state = 17,
                                    n_estimators = 100,
                                    max_features = "auto",
                                    min_samples_split = 0.03,
                                    min_samples_leaf = 0.01,
                                    max_depth = 10,
                                    max_leaf_nodes = None,
                                    n_jobs = 1))
    ]
)

pipe_TCAA_rf = Pipeline(
    [
        ('std_scaler', StandardScaler()),
        ("RF", RandomForestRegressor(random_state = 17,
                                    n_estimators = 100,
                                    max_features = "auto",
                                     min_samples_split = 2,
                                     min_samples_leaf = 1,
                                     max_depth = 30,
                                     max_leaf_nodes = None,
                                    n_jobs = 1))
    ]
)

In [ ]:
pipe_DCAA_svr = Pipeline(
    [
        ('std_scaler', StandardScaler()),
        ("SVR", SVR(C=6,
                    epsilon=0.0003,
                    gamma=0.003,
                    kernel='rbf',
                    max_iter=-1,
                    shrinking=True,
                    tol=0.001,
                    verbose=False))
    ]
)

pipe_TCAA_svr = Pipeline(
    [
        ('std_scaler', StandardScaler()),
        ("SVR", SVR(C=3,
                    epsilon=0.3,
                    gamma=0.001,
                    kernel='rbf',
                    max_iter=-1,
                    shrinking=True,
                    tol=0.001,
                    verbose=False))
    ]
)

In [ ]:
# max_list = [3,5,10,20,50,100,200,400,732]

In [ ]:
# max_list = [5,10,20,50,100]

In [ ]:
# max_list = [5,10, 20]

In [ ]:
max_list = [None]

In [ ]:
#### A new dictionary of selectors is defined using mse as scorer

selector_mse_DCAA_rf = dict()
selector_mse_DCAA_svr = dict()

for a in max_list:
    selector_mse_DCAA_rf[a] = GeneticSelectionCV(pipe_DCAA_rf,
                                  cv=5,
                                  verbose=1,
                                  scoring=rmse_scorer,
                                  max_features= a,
                                  n_population=1464,
                                  crossover_proba=0.5,
                                  mutation_proba=0.02,
                                  n_generations=10,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.02,
                                  tournament_size=5,
                                  n_gen_no_change=10,
                                  caching=True,
                                  n_jobs=-1)
    selector_mse_DCAA_svr[a] = GeneticSelectionCV(pipe_DCAA_svr,
                                  cv=5,
                                  verbose=1,
                                  scoring=rmse_scorer,
                                  max_features= a,
                                  n_population=1464,
                                  crossover_proba=0.5,
                                  mutation_proba=0.02,
                                  n_generations=55,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.02,
                                  tournament_size=5,
                                  n_gen_no_change=55,
                                  caching=True,
                                  n_jobs=-1)

In [ ]:
# A new dictionary of selectors is defined using mse as scorer

selector_mse_TCAA_rf = dict()
selector_mse_TCAA_svr = dict()

for a in max_list:
    selector_mse_TCAA_rf[a] = GeneticSelectionCV(pipe_TCAA_rf,
                                  cv=5,
                                  verbose=1,
                                  scoring=rmse_scorer,
                                  max_features= a,
                                  n_population=1464,
                                  crossover_proba=0.5,
                                  mutation_proba=0.02,
                                  n_generations=10,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.02,
                                  tournament_size=5,
                                  n_gen_no_change=10,
                                  caching=True,
                                  n_jobs=-1)
    
    selector_mse_TCAA_svr[a] = GeneticSelectionCV(pipe_TCAA_svr,
                                  cv=5,
                                  verbose=1,
                                  scoring=rmse_scorer,
                                  max_features= a,
                                  n_population=1464,
                                  crossover_proba=0.5,
                                  mutation_proba=0.02,
                                  n_generations=55,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.02,
                                  tournament_size=5,
                                  n_gen_no_change=55,
                                  caching=True,
                                  n_jobs=-1)

## Section 1 -  Random Forest

### Get the dictionaries of scores and the numbers of features selected

In [ ]:
rmse_scores_DCAA_rf = dict()
n_features_mse_DCAA_rf = dict()
generations_DCAA_rf = dict()


rmse_scores_TCAA_rf = dict()
n_features_mse_TCAA_rf = dict()
generations_TCAA_rf = dict()

for a in max_list:
    selector_mse_DCAA_rf[a] = selector_mse_DCAA_rf[a].fit(X, log_y[:,0])
    generations_DCAA_rf[a] = np.sqrt(-selector_mse_DCAA_rf[a].generation_scores_)
    rmse_scores_DCAA_rf[a] = np.sqrt(-selector_mse_DCAA_rf[a].generation_scores_[10])
    n_features_mse_DCAA_rf[a] = selector_mse_DCAA_rf[a].n_features_
    
    
    selector_mse_TCAA_rf[a] = selector_mse_TCAA_rf[a].fit(X,log_y[:,1])
    generations_TCAA_rf[a] = np.sqrt(-selector_mse_TCAA_rf[a].generation_scores_)
    rmse_scores_TCAA_rf[a] = np.sqrt(-selector_mse_TCAA_rf[a].generation_scores_[10])
    n_features_mse_TCAA_rf[a] = selector_mse_TCAA_rf[a].n_features_
    
    

    
    x_DCAA = np.array(list(n_features_mse_DCAA_rf.values())).astype(float)
    y_DCAA = np.array(list(rmse_scores_DCAA_rf.values())).astype(float)
    x_TCAA = np.array(list(n_features_mse_TCAA_rf.values())).astype(float)
    y_TCAA = np.array(list(rmse_scores_TCAA_rf.values())).astype(float)
    plt.scatter(x_DCAA, y_DCAA, color = "red")
    plt.scatter(x_TCAA, y_TCAA, color = "blue")
    plt.ylim(0.5,2)
    plt.ylabel("Cross-validation score (RMSE$_{CV}$)")
    plt.xlabel("Number of features selected")
    plt.legend(["DCAA", "TCAA"])
    
    plt.savefig("../Jupyter/results/figures/RF_" +str(a) + "descs_20200516.pdf")
    plt.show()


  

In [ ]:
#black and white
plt.scatter(x_DCAA, y_DCAA, color = "black", marker  = "^")
plt.scatter(x_TCAA, y_TCAA, color = "black")
plt.ylim(0.5,2)
plt.ylabel("Cross-validation score (RMSE$_{CV}$)")
plt.xlabel("Number of features selected")
plt.legend(["DCAA", "TCAA"])


plt.show()
plt.savefig("../Jupyter/results/figures/feature selection_B&W" + "RF_202005015.pdf")

In [ ]:
%store selector_mse_DCAA_rf
%store selector_mse_TCAA_rf

In [ ]:
%store rmse_scores_DCAA_rf
%store rmse_scores_TCAA_rf

%store generations_DCAA_rf
%store generations_TCAA_rf

%store n_features_mse_DCAA_rf
%store n_features_mse_TCAA_rf

## Section 2 - SVR

In [ ]:
rmse_scores_DCAA_svr = dict()
n_features_mse_DCAA_svr = dict()
generations_DCAA_svr = dict()


rmse_scores_TCAA_svr = dict()
n_features_mse_TCAA_svr = dict()
generations_TCAA_svr = dict()



for a in max_list:
    selector_mse_DCAA_svr[a] = selector_mse_DCAA_svr[a].fit(X, log_y[:,0])
    generations_DCAA_svr[a] = np.sqrt(-selector_mse_DCAA_svr[a].generation_scores_)
    rmse_scores_DCAA_svr[a] = np.sqrt(-selector_mse_DCAA_svr[a].generation_scores_[55])
    n_features_mse_DCAA_svr[a] = selector_mse_DCAA_svr[a].n_features_
    
    selector_mse_TCAA_svr[a] = selector_mse_TCAA_svr[a].fit(X, log_y[:,1])
    generations_TCAA_svr[a] = np.sqrt(-selector_mse_TCAA_svr[a].generation_scores_)
    rmse_scores_TCAA_svr[a] = np.sqrt(-selector_mse_TCAA_svr[a].generation_scores_[55])
    n_features_mse_TCAA_svr[a] = selector_mse_TCAA_svr[a].n_features_
    
    
    x_DCAA = np.array(list(n_features_mse_DCAA_svr.values())).astype(float)
    y_DCAA = np.array(list(rmse_scores_DCAA_svr.values())).astype(float)
    x_TCAA = np.array(list(n_features_mse_TCAA_svr.values())).astype(float)
    y_TCAA = np.array(list(rmse_scores_TCAA_svr.values())).astype(float)
    plt.scatter(x_DCAA, y_DCAA, color = "red")
    plt.scatter(x_TCAA, y_TCAA, color = "blue")
    plt.ylim(0.5,2)
    plt.ylabel("Cross-validation score (RMSE$_{CV}$)")
    plt.xlabel("Number of features selected")
    plt.legend(["DCAA", "TCAA"])
    
    plt.savefig("../Jupyter/results/figures/SVR_" +str(a) + "descs_20200516.pdf")
    plt.show()

 

In [ ]:
%store selector_mse_DCAA_svr
%store selector_mse_TCAA_svr

In [ ]:
#black and white
plt.scatter(x_DCAA, y_DCAA, color = "black", marker  = "^")
plt.scatter(x_TCAA, y_TCAA, color = "black")
plt.ylim(0.5,2)
plt.ylabel("Cross-validation score (RMSE$_{CV}$)")
plt.xlabel("Number of features selected")
plt.legend(["DCAA", "TCAA"])


plt.show()
plt.savefig("../Jupyter/results/figures/feature selection_B&W" + "SVR_20200516.pdf")

## Generations plots. Show that optimum solutions for both algorithms were found

In [ ]:
# SVR
# This plot shows the improvement in score over generations for different values of n_features.
# TCAA and DCAA are shown as subplots

fig, ax = plt.subplots(1, 2, sharex= True, sharey= True, figsize = (6,4))
# plt.ylabel("Cross-validation score (Q$^2$)")
# plt.xlabel("Number of generations")


for a in max_list:
    ax[0].plot(range(1, len(generations_DCAA_svr[a]) + 1), generations_DCAA_svr[a])
    ax[1].plot(range(1, len(generations_TCAA_svr[a]) + 1), generations_TCAA_svr[a])
    
    ax[1].legend(list(n_features_mse_DCAA_svr.values()), loc = "center left", bbox_to_anchor = (1,0.5))
    

    
plt.ylim(0.5,2) 

fig.text(0.5, 0.92, "Support Vector Regressor", ha='center')
fig.text(0.5, 0.004, "Number of generations", ha='center')
fig.text(0.04, 0.5, "Cross-validation score (RMSE$_{CV}$)", va='center', rotation='vertical')

plt.savefig("../Jupyter/results/figures/Genetic algorithm_SVR_rmsevsNgen_DCAA&TCAA_20200516.pdf")
plt.show()

In [ ]:
generations_DCAA_rf[a]

In [ ]:
# RF
# This plot shows the improvement in r2 over generations for different values of n_features.
# TCAA and DCAA are shown as subplots

fig, ax = plt.subplots(1, 2, sharex= True, sharey= True, figsize = (6,4))
# plt.ylabel("Cross-validation score (Q$^2$)")
# plt.xlabel("Number of generations")


for a in max_list:
    ax[0].plot(range(1, len(generations_DCAA_rf[a]) + 1), generations_DCAA_rf[a])
    ax[1].plot(range(1, len(generations_TCAA_rf[a]) + 1), generations_TCAA_rf[a])
    
    ax[1].legend(list(n_features_mse_DCAA_rf.values()), loc = "center left", bbox_to_anchor = (1,0.5))
    
plt.ylim(0.5,2) 

fig.text(0.5, 0.92, "Random Forest regressor", ha='center')
fig.text(0.5, 0.004, "Number of generations", ha='center')
fig.text(0.04, 0.5, "Cross-validation score (RMSE$_{CV}$)", va='center', rotation='vertical')

plt.savefig("../Jupyter/results/figures/Genetic algorithm_RF_rmsevsNgen_DCAA&TCAA_20200515.pdf")
plt.show()

### Get the dictionaries of scores and the numbers of features selected by GA

In [ ]:
#These variables allow to have the X_train and X_test filtered with the selected descriptors.
#Different subsets will allow different train and test sets (e.g. X_train_DCAA_rf)

features = list(df_qsar.columns.values)
X_train_df = pd.DataFrame(X_train)
X_train_df = X_train_df.rename(columns=dict(zip(X_train_df,features)))

X_test_df = pd.DataFrame(X_test)
X_test_df = X_test_df.rename(columns=dict(zip(X_test_df,features)))

In [ ]:
X_df = pd.DataFrame(X)
X_df = X_df.rename(columns=dict(zip(X_df,features)))

In [ ]:
# These are the descriptors selected by GA-RF as best predictors of DCAA

support_DCAA_rf = dict()
features_list_DCAA_rf = dict()

X_DCAA_rf = dict()
X_test_DCAA_rf = dict()
X_train_DCAA_rf = dict()

y_predicted_DCAA_rf = dict()
rmse_ext_DCAA_rf = dict()
r2_ext_DCAA_rf = dict()


for a in max_list:
    support_DCAA_rf[a] = selector_mse_DCAA_rf[a].support_
    support_DCAA_rf[a] = pd.DataFrame(support_DCAA_rf[a])
    support_DCAA_rf[a]["Features"] = features
    support_DCAA_rf[a].columns = ["Boolean", "Features"]
    support_DCAA_rf[a] = support_DCAA_rf[a][support_DCAA_rf[a]["Boolean"] == True]
    support_DCAA_rf[a] = support_DCAA_rf[a].drop(columns = "Boolean")
    features_list_DCAA_rf[a] = support_DCAA_rf[a]["Features"].tolist()
    
    X_DCAA_rf[a] = X_df.filter(features_list_DCAA_rf[a], axis = 1)     
    X_DCAA_rf[a] = X_DCAA_rf[a].values
    
    X_train_DCAA_rf[a] = X_train_df.filter(features_list_DCAA_rf[a], axis = 1)
    X_train_DCAA_rf[a] = X_train_DCAA_rf[a].values
    
    X_test_DCAA_rf[a] = X_test_df.filter(features_list_DCAA_rf[a], axis = 1)
    X_test_DCAA_rf[a] = X_test_DCAA_rf[a].values
    
    
    pipe_DCAA_rf.fit(X_train_DCAA_rf[a], y_train[:,0])
    y_predicted_DCAA_rf[a] = pipe_DCAA_rf.predict(X_test_DCAA_rf[a])
    rmse_ext_DCAA_rf[a] = metrics.mean_squared_error(y_test[:,0], y_predicted_DCAA_rf[a], squared = False)
    r2_ext_DCAA_rf[a] = metrics.r2_score(y_test[:,0], y_predicted_DCAA_rf[a])

In [ ]:
X_train_df.shape

In [ ]:
X_df.head(3)

In [ ]:
X_DCAA_rf[5].shape

In [ ]:
X_train_DCAA_rf[5].shape

In [ ]:
X_DCAA_svr[5].shape

In [ ]:
# These are the descriptors selected by GA-RF as best predictors of TCAA

support_TCAA_rf = dict()
features_list_TCAA_rf = dict()

X_TCAA_rf = dict()
X_test_TCAA_rf = dict()
X_train_TCAA_rf = dict()

y_predicted_TCAA_rf = dict()
rmse_ext_TCAA_rf = dict()
r2_ext_TCAA_rf = dict()


for a in max_list:
    support_TCAA_rf[a] = selector_mse_TCAA_rf[a].support_
    support_TCAA_rf[a] = pd.DataFrame(support_TCAA_rf[a])
    support_TCAA_rf[a]["Features"] = features
    support_TCAA_rf[a].columns = ["Boolean", "Features"]
    support_TCAA_rf[a] = support_TCAA_rf[a][support_TCAA_rf[a]["Boolean"] == True]
    support_TCAA_rf[a] = support_TCAA_rf[a].drop(columns = "Boolean")
    features_list_TCAA_rf[a] = support_TCAA_rf[a]["Features"].tolist()
    
    X_TCAA_rf[a] = X_df.filter(features_list_TCAA_rf[a], axis = 1)
    X_TCAA_rf[a] = X_TCAA_rf[a].values   
    
    X_train_TCAA_rf[a] = X_train_df.filter(features_list_TCAA_rf[a], axis = 1)
    X_train_TCAA_rf[a] = X_train_TCAA_rf[a].values
    
    X_test_TCAA_rf[a] = X_test_df.filter(features_list_TCAA_rf[a], axis = 1)
    X_test_TCAA_rf[a] = X_test_TCAA_rf[a].values
    
    pipe_TCAA_rf.fit(X_train_TCAA_rf[a], y_train[:,1])
    y_predicted_TCAA_rf[a] = pipe_TCAA_rf.predict(X_test_TCAA_rf[a])
    rmse_ext_TCAA_rf[a] = metrics.mean_squared_error(y_test[:,1], y_predicted_TCAA_rf[a], squared = False)
    r2_ext_TCAA_rf[a] = metrics.r2_score(y_test[:,1], y_predicted_TCAA_rf[a])

In [ ]:
print(r2_ext_DCAA_rf)
print(r2_ext_TCAA_rf)

In [ ]:
print(features_list_DCAA_rf)
print(features_list_TCAA_rf)

### Get the dictionaries of scores and the numbers of features selected by SVR

In [ ]:
# These are the descriptors selected by GA-SVR as best predictors of DCAA

support_DCAA_svr = dict()
features_list_DCAA_svr = dict()

X_DCAA_svr = dict()
X_test_DCAA_svr = dict()
X_train_DCAA_svr = dict()

y_predicted_DCAA_svr = dict()
rmse_ext_DCAA_svr = dict()
r2_ext_DCAA_svr = dict()


for a in max_list:
    support_DCAA_svr[a] = selector_mse_DCAA_svr[a].support_
    support_DCAA_svr[a] = pd.DataFrame(support_DCAA_svr[a])
    support_DCAA_svr[a]["Features"] = features
    support_DCAA_svr[a].columns = ["Boolean", "Features"]
    support_DCAA_svr[a] = support_DCAA_svr[a][support_DCAA_svr[a]["Boolean"] == True]
    support_DCAA_svr[a] = support_DCAA_svr[a].drop(columns = "Boolean")
    features_list_DCAA_svr[a] = support_DCAA_svr[a]["Features"].tolist()
    
    X_DCAA_svr[a] = X_df.filter(features_list_DCAA_svr[a], axis = 1)
    X_DCAA_svr[a] = X_DCAA_svr[a].values
    
    X_train_DCAA_svr[a] = X_train_df.filter(features_list_DCAA_svr[a], axis = 1)
    X_train_DCAA_svr[a] = X_train_DCAA_svr[a].values
    
    X_test_DCAA_svr[a] = X_test_df.filter(features_list_DCAA_svr[a], axis = 1)
    X_test_DCAA_svr[a] = X_test_DCAA_svr[a].values
    
    
    pipe_DCAA_svr.fit(X_train_DCAA_svr[a], y_train[:,0])
    y_predicted_DCAA_svr[a] = pipe_DCAA_svr.predict(X_test_DCAA_svr[a])
    rmse_ext_DCAA_svr[a] = metrics.mean_squared_error(y_test[:,0], y_predicted_DCAA_svr[a], squared = False)
    r2_ext_DCAA_svr[a] = metrics.r2_score(y_test[:,0], y_predicted_DCAA_svr[a])

In [ ]:
# These are the descriptors selected by GA-SVR as best predictors of TCAA

support_TCAA_svr = dict()
features_list_TCAA_svr = dict()

X_TCAA_svr = dict()
X_test_TCAA_svr = dict()
X_train_TCAA_svr = dict()

y_predicted_TCAA_svr = dict()
rmse_ext_TCAA_svr = dict()
r2_ext_TCAA_svr = dict()


for a in max_list:
    support_TCAA_svr[a] = selector_mse_TCAA_svr[a].support_
    support_TCAA_svr[a] = pd.DataFrame(support_TCAA_svr[a])
    support_TCAA_svr[a]["Features"] = features
    support_TCAA_svr[a].columns = ["Boolean", "Features"]
    support_TCAA_svr[a] = support_TCAA_svr[a][support_TCAA_svr[a]["Boolean"] == True]
    support_TCAA_svr[a] = support_TCAA_svr[a].drop(columns = "Boolean")
    features_list_TCAA_svr[a] = support_TCAA_svr[a]["Features"].tolist()
    
    X_TCAA_svr[a] = X_df.filter(features_list_TCAA_svr[a], axis = 1)
    X_TCAA_svr[a] = X_TCAA_svr[a].values
    
    X_train_TCAA_svr[a] = X_train_df.filter(features_list_TCAA_svr[a], axis = 1)
    X_train_TCAA_svr[a] = X_train_TCAA_svr[a].values
    
    X_test_TCAA_svr[a] = X_test_df.filter(features_list_TCAA_svr[a], axis = 1)
    X_test_TCAA_svr[a] = X_test_TCAA_svr[a].values
    
    
    pipe_TCAA_svr.fit(X_train_TCAA_svr[a], y_train[:,1])
    y_predicted_TCAA_svr[a] = pipe_TCAA_svr.predict(X_test_TCAA_svr[a])
    rmse_ext_TCAA_svr[a] = metrics.mean_squared_error(y_test[:,1], y_predicted_TCAA_svr[a], squared = False)
    r2_ext_TCAA_svr[a] = metrics.r2_score(y_test[:,1], y_predicted_TCAA_svr[a])

In [ ]:
print(r2_ext_DCAA_svr)
print(r2_ext_TCAA_svr)

In [ ]:
print(features_list_DCAA_svr)
print(features_list_TCAA_svr)

### Metrics for the linear models - Evaluate subsets of descriptors

In [ ]:
# dictionary of selectors 


DCAA_regressors = {"RF_opt" : RandomForestRegressor(random_state = 17,
                                    n_estimators = 300,
                                    max_features = "auto",
                                    min_samples_split = 0.03,
                                    min_samples_leaf = 0.01,
                                    max_depth = 10,
                                    max_leaf_nodes = None,
                                    n_jobs = -1),
              "SVR_{rbf}_opt" : SVR(C=6,
                                    epsilon=0.0003,
                                    gamma=0.003,
                                    kernel='rbf',
                                    max_iter=-1,
                                    shrinking=True,
                                    tol=0.001,
                                    verbose=False),
                   "SVR_{linear}" : svm.SVR(kernel = "linear"),
                   "MLP" : MLPRegressor(solver = "lbfgs", max_iter = 400, random_state = 17),
                   "MLR" : linear_model.LinearRegression()}

TCAA_regressors = {"RF_opt" : RandomForestRegressor(random_state = 17,
                                    n_estimators = 100,
                                    max_features = "auto",
                                    min_samples_split = 2,
                                    min_samples_leaf = 1,
                                    max_depth = 30,
                                    max_leaf_nodes = None),
              "SVR_{rbf}_opt" : SVR(C=3,
                                    epsilon=0.3,
                                    gamma=0.001,
                                    kernel='rbf',
                                    max_iter=-1,
                                    shrinking=True,
                                    tol=0.001,
                                    verbose=False),
                   "SVR_{linear}" : svm.SVR(kernel = "linear"),
                   "MLP" : MLPRegressor(solver = "lbfgs", max_iter = 400, random_state = 17),
                   "MLR" : linear_model.LinearRegression()}


In [ ]:
# Evaluate using the input provided by RF
# get scores for DCAA

y_pred_DCAA_rf = dict()

r2_DCAA_cv = dict()
rmse_DCAA_cv = dict()
r2_DCAA_ext = dict ()
rmse_DCAA_ext = dict()

scores_DCAA = dict()

reg_list_DCAA = DCAA_regressors.keys()

for a in reg_list_DCAA:
    for b in max_list:
        DCAA_regressors[a] = DCAA_regressors[a].fit(X_train_DCAA_rf[b], y_train[:,0])
        y_pred_DCAA_rf[b] = DCAA_regressors[a].predict(X_test_DCAA_rf[b])
    
#         r2_TCAA_cv[a] = model_selection.cross_validate(TCAA_regressors[a], X_train_scaled,y_train[:,1], scoring = "r2", cv =5 )
#         r2_TCAA_cv[a] = mean(r2_TCAA_cv[a]["test_score"])
#         rmse_TCAA_cv[a] = model_selection.cross_validate(TCAA_regressors[a], X_train_scaled,y_train[:,1], scoring = "neg_root_mean_squared_error", cv =5 )
#         rmse_TCAA_cv[a] = -mean(rmse_TCAA_cv[a]["test_score"])
        r2_DCAA_ext[b] = DCAA_regressors[a].score(X_test_DCAA_rf[b], y_test[:,0])
#         rmse_TCAA_ext[a] = sqrt(mean_squared_error(y_test[:,1], (y_pred_TCAA[a])))
        
#         print("n_descs: {:.5f}, R2: {:.5f}".format(max_list[b].keys, r2_DCAA_ext[b]))
        print(str(a) + "_" + str(b))
        print(r2_DCAA_ext[b])
        #scores = dict(zip(reg_list_DCAA[a], r2_DCAA_ext[b]))



In [ ]:
# Evaluate using the input provided by RF
# get scores for TCAA

y_pred_TCAA_rf = dict()

r2_TCAA_cv = dict()
rmse_TCAA_cv = dict()
r2_TCAA_ext = dict ()
rmse_TCAA_ext = dict()

scores_TCAA = dict()

reg_list_TCAA = TCAA_regressors.keys()

for a in reg_list_TCAA:
    for b in max_list:
        TCAA_regressors[a] = TCAA_regressors[a].fit(X_train_TCAA_rf[b], y_train[:,1])
        y_pred_TCAA_rf[b] = TCAA_regressors[a].predict(X_test_TCAA_rf[b])
    
#         r2_TCAA_cv[a] = model_selection.cross_validate(TCAA_regressors[a], X_train_scaled,y_train[:,1], scoring = "r2", cv =5 )
#         r2_TCAA_cv[a] = mean(r2_TCAA_cv[a]["test_score"])
#         rmse_TCAA_cv[a] = model_selection.cross_validate(TCAA_regressors[a], X_train_scaled,y_train[:,1], scoring = "neg_root_mean_squared_error", cv =5 )
#         rmse_TCAA_cv[a] = -mean(rmse_TCAA_cv[a]["test_score"])
        r2_TCAA_ext[b] = TCAA_regressors[a].score(X_test_TCAA_rf[b], y_test[:,1])
#         rmse_TCAA_ext[a] = sqrt(mean_squared_error(y_test[:,1], (y_pred_TCAA[a])))
        
#         print("n_descs: {:.5f}, R2: {:.5f}".format(max_list[b].keys, r2_DCAA_ext[b]))
        print(str(a) + "_" + str(b))
        print(r2_TCAA_ext[b])
        #scores = dict(zip(reg_list_DCAA[a], r2_DCAA_ext[b]))

## Evaluate using GA-SVR subset as input

In [ ]:
# Evaluate using the input provided by SVR
# get scores for DCAA

y_pred_DCAA_svr = dict()

r2_DCAA_cv = dict()
rmse_DCAA_cv = dict()
r2_DCAA_ext = dict ()
rmse_DCAA_ext = dict()

scores_DCAA = dict()

reg_list_DCAA = DCAA_regressors.keys()

for a in reg_list_DCAA:
    for b in max_list:
        DCAA_regressors[a] = DCAA_regressors[a].fit(X_train_DCAA_svr[b], y_train[:,0])
        y_pred_DCAA_svr[b] = DCAA_regressors[a].predict(X_test_DCAA_svr[b])
    
#         r2_TCAA_cv[a] = model_selection.cross_validate(TCAA_regressors[a], X_train_scaled,y_train[:,1], scoring = "r2", cv =5 )
#         r2_TCAA_cv[a] = mean(r2_TCAA_cv[a]["test_score"])
#         rmse_TCAA_cv[a] = model_selection.cross_validate(TCAA_regressors[a], X_train_scaled,y_train[:,1], scoring = "neg_root_mean_squared_error", cv =5 )
#         rmse_TCAA_cv[a] = -mean(rmse_TCAA_cv[a]["test_score"])
        r2_DCAA_ext[b] = DCAA_regressors[a].score(X_test_DCAA_svr[b], y_test[:,0])
#         rmse_TCAA_ext[a] = sqrt(mean_squared_error(y_test[:,1], (y_pred_TCAA[a])))
        
#         print("n_descs: {:.5f}, R2: {:.5f}".format(max_list[b].keys, r2_DCAA_ext[b]))
        print(str(a) + "_" + str(b))
        print(r2_DCAA_ext[b])
        #scores = dict(zip(reg_list_DCAA[a], r2_DCAA_ext[b]))


In [ ]:
# Evaluate using the input provided by SVR
# get scores for TCAA

y_pred_TCAA_svr = dict()

r2_TCAA_cv = dict()
rmse_TCAA_cv = dict()
r2_TCAA_ext = dict ()
rmse_TCAA_ext = dict()

scores_TCAA = dict()

reg_list_TCAA = TCAA_regressors.keys()

for a in reg_list_TCAA:
    for b in max_list:
        TCAA_regressors[a] = TCAA_regressors[a].fit(X_train_TCAA_svr[b], y_train[:,1])
        y_pred_TCAA_svr[b] = TCAA_regressors[a].predict(X_test_TCAA_svr[b])
    
#         r2_TCAA_cv[a] = model_selection.cross_validate(TCAA_regressors[a], X_train_scaled,y_train[:,1], scoring = "r2", cv =5 )
#         r2_TCAA_cv[a] = mean(r2_TCAA_cv[a]["test_score"])
#         rmse_TCAA_cv[a] = model_selection.cross_validate(TCAA_regressors[a], X_train_scaled,y_train[:,1], scoring = "neg_root_mean_squared_error", cv =5 )
#         rmse_TCAA_cv[a] = -mean(rmse_TCAA_cv[a]["test_score"])
        r2_TCAA_ext[b] = TCAA_regressors[a].score(X_test_TCAA_svr[b], y_test[:,1])
#         rmse_TCAA_ext[a] = sqrt(mean_squared_error(y_test[:,1], (y_pred_TCAA[a])))
        
#         print("n_descs: {:.5f}, R2: {:.5f}".format(max_list[b].keys, r2_DCAA_ext[b]))
        print(str(a) + "_" + str(b))
        print(r2_TCAA_ext[b])
        #scores = dict(zip(reg_list_DCAA[a], r2_DCAA_ext[b]))

### Test LOO-CV

In [ ]:
X_DCAA_rf[20].shape

In [ ]:

# Evaluate LOO-Q2 in all regressors using the input provided by RF


reg_list_DCAA = DCAA_regressors.keys()


# calculate the leave one out cross validation for DCAA


loo_q2_DCAA = dict()
loo_rmse_DCAA = dict ()

y_tests_nested_dict = dict()
y_preds_nested_dict = dict()

y_tests_dict = dict()
y_preds_dict = dict()


reg_list_DCAA = DCAA_regressors.keys()

yr = log_y[:,0] #y is the same for every regressor, just remember to differentiate TCAA and DCAA. 

for a in reg_list_DCAA:
    for b in max_list:
        Xr = X_DCAA_rf[b] #the key is to define a different Xr per subset of descs X_train_DCAA_rf[b] 
        loo = LeaveOneOut()
        ytests = []
        ypreds = []
        for train_idx, test_idx in loo.split(Xr):
            X_train, X_test = Xr[train_idx], Xr[test_idx] #requires arrays
            y_train, y_test = yr[train_idx], yr[test_idx]
    
            #models
            DCAA_regressors[a].fit(X = X_train, y = y_train) 
            y_pred = DCAA_regressors[a].predict(X_test)
        
        # there is only one y-test and y-pred per iteration over the loo.split, 
        # so to get a proper graph, we append them to respective lists.
        
            ytests += list(y_test)
            ypreds += list(y_pred)
        
        y_tests_dict[b] = ytests
        y_preds_dict[b] = ypreds
        
        
        loo_q2_DCAA[b] = metrics.r2_score(ytests, ypreds)
        loo_rmse_DCAA[b] = metrics.mean_squared_error(ytests, ypreds, squared = False)
        
        print(str(a) + "_" + str(b))
        print("Leave One Out Cross Validation" + str(a))
        print("LOO $Q^2$: {:.5f}, MSE: {:.5f}".format(loo_q2_DCAA[b], loo_rmse_DCAA[b]))
    y_tests_nested_dict[a] = {str(b): y_tests_dict[b]}
    y_preds_nested_dict[a] = {str(b): y_preds_dict[b]}

In [ ]:

# Evaluate LOO-Q2 in all regressors using the input provided by RF


reg_list_TCAA = TCAA_regressors.keys()


# calculate the leave one out cross validation for TCAA


loo_q2_TCAA = dict()
loo_rmse_TCAA = dict ()

y_tests_nested_dict_TCAA = dict()
y_preds_nested_dict_TCAA = dict()

y_tests_dict_TCAA = dict()
y_preds_dict_TCAA = dict()



yr = log_y[:,1] #y is the same for every regressor, just remember to differentiate TCAA and DCAA. 

for a in reg_list_TCAA:
    for b in max_list:
        Xr = X_TCAA_rf[b] #the key is to define a different Xr per subset of descs X_train_DCAA_rf[b] 
        loo = LeaveOneOut()
        ytests = []
        ypreds = []
        for train_idx, test_idx in loo.split(Xr):
            X_train, X_test = Xr[train_idx], Xr[test_idx] #requires arrays
            y_train, y_test = yr[train_idx], yr[test_idx]
    
            #models
            TCAA_regressors[a].fit(X = X_train, y = y_train) 
            y_pred = TCAA_regressors[a].predict(X_test)
        
        # there is only one y-test and y-pred per iteration over the loo.split, 
        # so to get a proper graph, we append them to respective lists.
        
            ytests += list(y_test)
            ypreds += list(y_pred)
        
        y_tests_dict_TCAA[b] = ytests
        y_preds_dict_TCAA[b] = ypreds
        
        
        loo_q2_TCAA[b] = metrics.r2_score(ytests, ypreds)
        loo_rmse_TCAA[b] = metrics.mean_squared_error(ytests, ypreds, squared = False)
        
        print(str(a) + "_" + str(b))
        print("Leave One Out Cross Validation" + str(a))
        print("LOO $Q^2$: {:.5f}, MSE: {:.5f}".format(loo_q2_TCAA[b], loo_rmse_TCAA[b]))
    y_tests_nested_dict_TCAA[a] = {str(b): y_tests_dict_TCAA[b]}
    y_preds_nested_dict_TCAA[a] = {str(b): y_preds_dict_TCAA[b]}

### Input of SVR

In [ ]:
# Evaluate LOO-Q2 in all regressors using the input provided by RF


reg_list_TCAA = TCAA_regressors.keys()


# calculate the leave one out cross validation for TCAA


loo_q2_TCAA = dict()
loo_rmse_TCAA = dict ()

y_tests_nested_dict_TCAA = dict()
y_preds_nested_dict_TCAA = dict()

y_tests_dict_TCAA = dict()
y_preds_dict_TCAA = dict()



yr = log_y[:,1] #y is the same for every regressor, just remember to differentiate TCAA and DCAA. 

for a in reg_list_TCAA:
    for b in max_list:
        Xr = X_TCAA_svr[b] #the key is to define a different Xr per subset of descs X_train_DCAA_rf[b] 
        loo = LeaveOneOut()
        ytests = []
        ypreds = []
        for train_idx, test_idx in loo.split(Xr):
            X_train, X_test = Xr[train_idx], Xr[test_idx] #requires arrays
            y_train, y_test = yr[train_idx], yr[test_idx]
    
            #models
            TCAA_regressors[a].fit(X = X_train, y = y_train) 
            y_pred = TCAA_regressors[a].predict(X_test)
        
        # there is only one y-test and y-pred per iteration over the loo.split, 
        # so to get a proper graph, we append them to respective lists.
        
            ytests += list(y_test)
            ypreds += list(y_pred)
        
        y_tests_dict_TCAA[b] = ytests
        y_preds_dict_TCAA[b] = ypreds
        
        
        loo_q2_TCAA[b] = metrics.r2_score(ytests, ypreds)
        loo_rmse_TCAA[b] = metrics.mean_squared_error(ytests, ypreds, squared = False)
        
        print(str(a) + "_" + str(b))
        print("Leave One Out Cross Validation" + str(a))
        print("LOO $Q^2$: {:.5f}, MSE: {:.5f}".format(loo_q2_TCAA[b], loo_rmse_TCAA[b]))
    y_tests_nested_dict_TCAA[a] = {str(b): y_tests_dict_TCAA[b]}
    y_preds_nested_dict_TCAA[a] = {str(b): y_preds_dict_TCAA[b]}

In [ ]:

# Evaluate LOO-Q2 in all regressors using the input provided by SVR


reg_list_DCAA = DCAA_regressors.keys()


# calculate the leave one out cross validation for DCAA


loo_q2_DCAA = dict()
loo_rmse_DCAA = dict ()

y_tests_nested_dict = dict()
y_preds_nested_dict = dict()

y_tests_dict = dict()
y_preds_dict = dict()


reg_list_DCAA = DCAA_regressors.keys()

yr = log_y[:,0] #y is the same for every regressor, just remember to differentiate TCAA and DCAA. 

for a in reg_list_DCAA:
    for b in max_list:
        Xr = X_DCAA_svr[b] #the key is to define a different Xr per subset of descs X_train_DCAA_rf[b] 
        loo = LeaveOneOut()
        ytests = []
        ypreds = []
        for train_idx, test_idx in loo.split(Xr):
            X_train, X_test = Xr[train_idx], Xr[test_idx] #requires arrays
            y_train, y_test = yr[train_idx], yr[test_idx]
    
            #models
            DCAA_regressors[a].fit(X = X_train, y = y_train) 
            y_pred = DCAA_regressors[a].predict(X_test)
        
        # there is only one y-test and y-pred per iteration over the loo.split, 
        # so to get a proper graph, we append them to respective lists.
        
            ytests += list(y_test)
            ypreds += list(y_pred)
        
        y_tests_dict[b] = ytests
        y_preds_dict[b] = ypreds
        
        
        loo_q2_DCAA[b] = metrics.r2_score(ytests, ypreds)
        loo_rmse_DCAA[b] = metrics.mean_squared_error(ytests, ypreds, squared = False)
        
        print(str(a) + "_" + str(b))
        print("Leave One Out Cross Validation" + str(a))
        print("LOO $Q^2$: {:.5f}, MSE: {:.5f}".format(loo_q2_DCAA[b], loo_rmse_DCAA[b]))
    y_tests_nested_dict[a] = {str(b): y_tests_dict[b]}
    y_preds_nested_dict[a] = {str(b): y_preds_dict[b]}

## Get the data of features selected as tables


In [ ]:
support_ga_DCAA = selector_mse_DCAA_rf[10].support_
support_ga_TCAA = selector_mse_TCAA_rf[10].support_

df_support_ga_DCAA = pd.DataFrame(support_ga_DCAA)
df_support_ga_TCAA = pd.DataFrame(support_ga_TCAA)

#### Get a list of the names of all features in order to extract the name of the selected descs from df_support

In [ ]:
# list first column of qsar_transposed as "features"  to get the names
features = list(df_qsar.columns.values)
len(features)
# join df_qsar_transposed  and df_support
df_support_ga_DCAA["Features"] = features
df_support_ga_DCAA.columns = ["Boolean", "Features"]
# extract only true values
df_support_ga_DCAA_true = df_support_ga_DCAA[df_support_ga_DCAA["Boolean"] == True]
#drop the "Boolean" and convert to Latex including feature number
df_support_ga_DCAA_true_only = df_support_ga_DCAA_true.drop(columns = "Boolean")
print(df_support_ga_DCAA_true_only.to_latex(index=True))

In [ ]:
# list first column of qsar_transposed as "features"  to get the names
features = list(df_qsar.columns.values)
len(features)
# join df_qsar_transposed  and df_support
df_support_ga_TCAA["Features"] = features
df_support_ga_TCAA.columns = ["Boolean", "Features"]
# extract only true values
df_support_ga_TCAA_true = df_support_ga_TCAA[df_support_ga_TCAA["Boolean"] == True]
#drop the "Boolean" and convert to Latex including feature number
df_support_ga_TCAA_true_only = df_support_ga_TCAA_true.drop(columns = "Boolean")
print(df_support_ga_TCAA_true_only.to_latex(index=True))

In [ ]:
# list first column of qsar_transposed as "features"  to get the names
features = list(df_qsar.columns.values)
len(features)
# join df_qsar_transposed  and df_support
df_support_ga["Features"] = features
df_support_ga.columns = ["Boolean", "Features"]
# extract only true values
df_support_ga_true = df_support_ga[df_support_ga["Boolean"] == True]
#drop the "Boolean" and convert to Latex including feature number
df_support_ga_true_only = df_support_ga_true.drop(columns = "Boolean")
print(df_support_ga_true_only.to_latex(index=True))

\begin{array}{cc}
\hline Feature\:number & Name  \\\hline
103 &         ATS8p \\
114 &       AATS1dv \\
131 &        AATS2v \\
152 &        AATS3i \\
190 &        ATSC1m \\
208 &       ATSC1se \\
209 &       ATSC2se \\
341 &        GATS1m \\
369 &       nBondsA \\
378 &         C2SP2 \\
386 &        Xch-3d \\
404 &        Xpc-4d \\
413 &         Xp-4d \\
460 &         NaasC \\
508 &          NdSe \\
535 &        SsssCH \\
613 &          TIC0 \\
634 &          CIC3 \\
653 &  FilterItLogS \\
693 &   EState\_VSA5 \\
723 &         piPC6 \\
731 &         nRing \\
782 &       n5ARing \\
888 &          JGI4 \\
926 &       Zagreb1 \\
\end{array}

## Selecting descriptors using RFECV-RF

##### Define estimator( regression model). In case it is necessary for the wrapping method.

In [ ]:
class PipelineRFE(Pipeline):

    def fit(self, X, y=None, **fit_params):
        super(PipelineRFE, self).fit(X, y, **fit_params)
        self.feature_importances_ = self.steps[-1][-1].feature_importances_
        return self

In [ ]:
pipe = PipelineRFE(
    [
        ('std_scaler', StandardScaler()),
        ("RF", RandomForestRegressor(random_state = 17))
    ]
)

##### Define selector (wrapping method)

In [ ]:
from sklearn.feature_selection import RFECV

In [ ]:
selector = RFECV(pipe, step = 1, cv = 5, scoring=rmse_scorer, verbose = True, n_jobs = -1)

In [ ]:
selector_RFECV_DCAA = selector.fit(X,log_y[:,0])     #currently y = DCAAFP

In [ ]:
selector_RFECV_TCAA = selector.fit(X,log_y[:,1])     #currently y = TCAAFP

In [ ]:
DCAA_features = selector_RFECV_DCAA.n_features_
print (DCAA_features)
TCAA_features = selector_RFECV_TCAA.n_features_
print (TCAA_features)

#### Create the number of features vs score graph

In [ ]:
cv_grid_rmse_DCAA = np.sqrt(-selector_RFECV_DCAA.grid_scores_)

In [ ]:
cv_grid_rmse_TCAA = np.sqrt(-selector_RFECV_TCAA.grid_scores_)

In [ ]:
# DCAA plot

# Plot number of features VS. cross-validation scores              #square root
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score ")
plt.plot(range(1, len(cv_grid_rmse_DCAA) + 1), cv_grid_rmse_DCAA)

plt.savefig("../Jupyter/results/figures/RFECV_RF_rmsevsNgen_DCAA_20200516.pdf")
plt.show()

In [ ]:
# TCAA plot

# Plot number of features VS. cross-validation scores              #square root
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score ")
plt.plot(range(1, len(cv_grid_rmse_TCAA) + 1), cv_grid_rmse_TCAA)

plt.savefig("../Jupyter/results/figures/RFECV_RF_rmsevsNgen_TCAA_20200516.pdf")
plt.show()

## Get the features selected by the RFECV


In [ ]:
support_rfecv_DCAA = selector_RFECV_DCAA.support_
support_rfecv_TCAA = selector_RFECV_TCAA.support_

In [ ]:
df_support_rfecv = pd.DataFrame(support_rfecv)

#### Get a list of the names of all features in order to extract the name of the selected descs from df_support

In [ ]:
# list first column of qsar_transposed as "features"  to get the names
features = list(df_qsar.columns.values)
len(features)

In [ ]:
# join df_qsar_transposed  and df_support
df_support_rfecv["Features"] = features                          #include features
df_support_rfecv.columns = ["Boolean", "Features"]               # rename column as Boolean
df_support_rfecv_true = df_support_rfecv[df_support_rfecv["Boolean"] == True]    # extract only true values
df_support_rfecv_true                           #show head and shape of the "True features" table

# Test the simplified model

#### Train the model

In [ ]:
# get the list of features (features_rfecv or features_ga) and use it to filter and get the simplified input
#df_qsar_rfecv or df_qsar_rfecv
#this new data frame can be used as input in other notebooks if stored with an appropiate name
#it can be used to test all the preliminary models by creating a copy 

In [ ]:
rfecv_features_list = df_support_rfecv_true["Features"].tolist() 
ga_features_list = df_support_ga_true["Features"].tolist() 

In [ ]:
#df_qsar_new contans only the descriptors selected by RFECV
df_qsar_rfecv = df_qsar.filter(rfecv_features_list, axis = 1)
df_qsar_ga = df_qsar.filter(ga_features_list, axis = 1)

In [ ]:
df_qsar_ga

In [ ]:
# #df_qsar_new contans only the descriptors selected by RFECV
# df_qsar_rfecv = df_qsar.filter(["ATS8m", "ATSC1are", "ATSC2i", "ZMIC1", "ZMIC2", "SlogP_VSA10"], axis = 1)

In [ ]:
df_qsar_rfecv

In [ ]:
%store df_qsar_ga
%store df_qsar_rfecv